In [1]:
import pandas as pd
import json
import requests
from pandas import json_normalize
#import ebmdatalab
from ebmdatalab import bq
import os

In [2]:
res = requests.get('https://api.github.com/repos/ebmdatalab/openprescribing/contents/openprescribing/measures/definitions') #uses GitHub API to get list of all files listed in measure definitions - defaults to main branch
data = res.text #creates text from API result
df = pd.read_json(data) #turns JSON from API result into dataframe

In [3]:
json_df = pd.DataFrame() #creates blank dataframe
for row in df[df['name'].str.contains('.json')].itertuples(index=True): #iterates through rows, and continues if file is .json
        url = (getattr(row, "download_url")) #gets URL from API request  
        data = json.loads(requests.get(url).text) #gets JSON from measure definition URL
        norm_df = pd.json_normalize(data, max_level=1) #normalises measure definition JSON into dataframe
        json_df = pd.concat([json_df,norm_df], axis=0, ignore_index=True) # concatentates into single dataframe
        json_df['file_name'] = 'ccg_data_' + df['name'].str.split('.').str[0].copy()

In [4]:
tags_df = json_df.explode('tags')
core_df = tags_df[['file_name', 'tags', 'radar_exclude']].copy()
filtered_core_df = core_df[((core_df['tags'].str.contains('core')) | (core_df['tags'].str.contains('lowpriority'))) & (core_df['radar_exclude'] != 'True')]

In [5]:
display(filtered_core_df)

,file_name,tags,radar_exclude
0,ccg_data_aafpercent,core,NaN
1,ccg_data_amox_500_15_caps,core,False
2,ccg_data_bdzadq,core,NaN
3,ccg_data_bdzper1000,core,NaN
4,ccg_data_carbon_salbutamol,core,False
...,...,...,...
65,ccg_data_solublepara,core,NaN
66,ccg_data_statinintensity,core,NaN
68,ccg_data_toothpaste,core,NaN
71,ccg_data_uti_antibiotics_3_day,core,NaN


In [ ]:
#create for next loop to go through each table name in the previous query
for name in filtered_core_df['file_name']:
    
    sql = """
    SELECT
      month, 
      pct_id as code, 
      numerator, 
      denominator, 
    FROM
      `ebmdatalab.measures.{}` AS a
    """
    
    sql = sql.format(name) #using python string to add table_name to SQL
    #concatenate each table name into single file during for next loop
    bq.cached_read(sql, os.path.join("..", "data", "test", "{}", "bq_cache.csv").format(name), use_cache=False)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=725825577420-unm2gnkiprugilg743tkbig250f4sfsj.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=sekWra1pmC6S80kebGnVn9nBOTc3yg&prompt=consent&access_type=offline
